## Patch based approach

Dataset

In [ ]:
import os
import numpy as np
import face_recognition

LABELS = ['22q11','Angelman','Apert','CDL','Down','FragileX','Marfan','Progeria','Sotos','TreacherCollins','Turner','Williams']


def load_dataset_patches(setPath):
    X = []
    y = []

    for label in LABELS:
        directory = os.path.join(setPath,label)
        for f in os.listdir(directory):
            path = os.path.join(directory,f)
            
            if os.path.isfile(path) and f != 'desktop.ini':
                
                # Extract landmarks
                image = face_recognition.load_image_file(path)
                face_landmarks_list = face_recognition.face_landmarks(image)

                # TODO need to extract a patch from the image instead of the keypoint itself
                
                # Iterate through detected faces
                keypoints = []
                for face in face_landmarks_list:

                    # Get face center
                    # and convert points into list form
                    cX = 0
                    cY = 0
                    count = 0
                    points = []

                    for featureType in face.keys():
                        for point in face[featureType]:
                            points = points + list(point)

                            cY = cY + point[0]
                            cX = cX + point[1]
                            count = count + 1
                    cX = cX / count
                    cY = cY / count

                    # Center all points
                    for i in range(0,len(points)):
                        if i % 2 == 0:
                            points[i] = points[i] - cY
                        else:
                            points[i] = points[i] - cX

                    # Store result
                    keypoints.append(points)

                # Append to dataset
                for facePoints in keypoints:
                    X.append(facePoints)
                    y.append(label)
    
    return np.array(X), np.array(y)

In [ ]:
X_test, y_test = load_dataset('./data/raw/Test')
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train, y_train = load_dataset('./data/raw/Train')
print(X_train.shape)
print(y_train.shape)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
Y_train = encoder.fit_transform(y_train)
Y_test = encoder.transform(y_test)

Model

In [ ]:
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten

model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(32,32,72)))

Train

Evaluate